In [1]:
import numpy as np
import pandas as pd
import jittor as jt
import os
from os.path import join, getsize

from jittor.dataset import Dataset
from PIL import Image
from dataset import Tiny_vid
import matplotlib.pyplot as plt



[i 0419 16:44:58.152588 60 log.cc:351] Load log_sync: 1
[i 0419 16:44:58.276594 60 compiler.py:951] Jittor(1.3.2.5) src: /opt/miniconda/envs/dhy/lib/python3.9/site-packages/jittor
[i 0419 16:44:58.284781 60 compiler.py:952] g++ at /usr/bin/g++(9.3.0)
[i 0419 16:44:58.286829 60 compiler.py:953] cache_path: /home/dhy/.cache/jittor/jt1.3.2/g++9.3.0/py3.9.7/Linux-3.10.0-1xeb/IntelRXeonRGolx19/default
[i 0419 16:44:58.301499 60 install_cuda.py:51] cuda_driver_version: [11, 4]
[i 0419 16:44:58.309968 60 __init__.py:411] Found /home/dhy/.cache/jittor/jtcuda/cuda11.2_cudnn8_linux/bin/nvcc(11.2.152) at /home/dhy/.cache/jittor/jtcuda/cuda11.2_cudnn8_linux/bin/nvcc.
[i 0419 16:44:58.360354 60 __init__.py:411] Found gdb(9.2) at /usr/bin/gdb.
[i 0419 16:44:58.368725 60 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.
[i 0419 16:44:58.514514 60 compiler.py:1006] cuda key:cu11.2.152_sm_86
[i 0419 16:44:58.736182 60 __init__.py:227] Total mem: 251.43GB, using 16 procs for compiling.
[i 04

In [2]:
from jittor import nn
import jittor.transform as trans
import numpy as np
from jittor.models.resnet import Resnet50, ResNet, Resnet34
from dataset import Tiny_vid
from argparse import ArgumentParser
from bbox import box_iou_batch
jt.misc.set_global_seed(425)

In [3]:
def bbox_to_rect(bbox, color):
    # 将边界框(左上x,左上y,右下x,右下y)格式转换成matplotlib格式：
    # ((左上x,左上y),宽,高)
    return plt.Rectangle(
        xy=(bbox[0], bbox[1]), width=bbox[2]-bbox[0], height=bbox[3]-bbox[1],
        fill=False, edgecolor=color, linewidth=2)

In [4]:
# dataset = Tiny_vid(train=False).set_attrs(batch_size=1, shuffle=True)
# class_dict = {'bird':0.,'car':1.,'dog':2.,'lizard':3.,'turtle':4.}
# from IPython import display
# for x, y in dataset:
#     plt.figure()
#     ax = plt.imshow(x.squeeze(0))
#     test_bbox = np.floor(y[1]*128)
#     test_bbox = test_bbox.squeeze(0).tolist()
#     ax.axes.add_patch(bbox_to_rect(test_bbox, 'blue'))
#     plt.title(f"'bird':0.,'car':1.,'dog':2.,'lizard':3.,'turtle':4.class:{y[0]}")    
#     plt.show()
#     a = input("aa:")
#     if a == '1':
#         display.clear_output(wait=True)
#         continue
#     if a == '2':
#         break

In [5]:
def box_iou(boxes1, boxes2):
    """计算两个锚框或边界框列表中成对的交并比"""
    box_area = lambda boxes: ((boxes[:, 2] - boxes[:, 0]) *
                              (boxes[:, 3] - boxes[:, 1]))
    # boxes1,boxes2,areas1,areas2的形状:
    # boxes1：(boxes1的数量,4),
    # boxes2：(boxes2的数量,4),
    # areas1：(boxes1的数量,),
    # areas2：(boxes2的数量,)
    areas1 = box_area(boxes1)
    areas2 = box_area(boxes2)
    import pdb;pdb.set_trace()

    # inter_upperlefts,inter_lowerrights,inters的形状:
    # (boxes1的数量,boxes2的数量,2)
    inter_upperlefts = np.maximum(boxes1[:, None, :2], boxes2[:, :2])
    inter_lowerrights = np.minimum(boxes1[:, None, 2:], boxes2[:, 2:])
    inters = (inter_lowerrights - inter_upperlefts).clip(min=0)
    # inter_areasandunion_areas的形状:(boxes1的数量,boxes2的数量)
    inter_areas = inters[:, :, 0] * inters[:, :, 1]
    union_areas = areas1[:, None] + areas2 - inter_areas
    return inter_areas / union_areas

In [6]:
class DetNet(nn.Module):
    def __init__(self):
        super(DetNet, self).__init__()
        self.preprocess = nn.Sequential(
            nn.Relu(),
            nn.AdaptiveAvgPool2d((1,1)),
            
        )
        # self.backbone = Resnet50(num_classes=5, pretrained=True)
        # self.backbone = Resnet50(pretrained=True)
        resnet = Resnet34(pretrained=True)
        layers = list(resnet.children())[:8]
        self.backbone = nn.Sequential(*layers)
        # pdb.set_trace()
        
        self.class_head = nn.Sequential(
            nn.BatchNorm1d(512),
            nn.Dropout(),
            nn.Linear(512,5)
        )
        self.bbox_head = nn.Sequential(
            nn.BatchNorm1d(512),
            nn.Linear(512,4)
        )
        
        
    def execute(self, x):
        # pdb.set_trace()
        # with jt.no_grad():
        y = self.backbone(x)
        y = self.preprocess(y)
        y =  y.view(y.shape[0], -1)
        class_score = self.class_head(y)
        bbox_out = self.bbox_head(y)
        return class_score, bbox_out



def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # import pdb;pdb.set_trace()
        outputs, outboxes = model(inputs)
        with jt.no_grad():
            batch_size = inputs.shape[0]
            pred,_ = jt.argmax(outputs.data, dim=1)
            class_acc = jt.sum(targets[0] == pred) / batch_size
            iou = box_iou_batch(outboxes, targets[1])
            all_correct = jt.sum((iou > 0.5)*(targets[0] == pred))
        loss_b = nn.l1_loss(outboxes, targets[1])
        # loss_b = loss_b.sum()
        loss_c = nn.cross_entropy_loss(outputs, targets[0], reduction="mean")
        # pdb.set_trace()
        loss = loss_c+ loss_b
        optimizer.step (loss)
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tClass Acc: {:.6f}\tBox_Acc: {:.6f}\tAll Acc: {:.6f}'.format(
                    epoch, batch_idx, len(train_loader),
                    100. * batch_idx / len(train_loader), loss.data[0], class_acc, jt.sum(iou > 0.5)/batch_size, all_correct/batch_size))



def train_class_head(model, train_loader, optimizer, epoch):
    model.train()
    # freeze bbox head
    for name, param in model.class_head.named_parameters():
        param.requires_grad = True
    for name, param in model.backbone.named_parameters():
        param.requires_grad = True
    for name, param in model.preprocess.named_parameters():
        param.requires_grad = True
    for name, param in model.bbox_head.named_parameters():
        param.requires_grad = False
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # import pdb;pdb.set_trace()
        outputs, outboxes = model(inputs)
        with jt.no_grad():
            batch_size = inputs.shape[0]
            pred,_ = jt.argmax(outputs.data, dim=1)
            class_acc = jt.sum(targets[0] == pred) / batch_size
        loss = nn.cross_entropy_loss(outputs, targets[0])
        optimizer.step (loss)
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTrain Acc: {:.6f}'.format(
                    epoch, batch_idx, len(train_loader),
                    100. * batch_idx / len(train_loader), loss.data[0], class_acc))


def train_bbox_head(model, train_loader, optimizer, epoch):
    model.train()
    # freeze preprocess and class head
    for name, param in model.class_head.named_parameters():
        param.requires_grad = False
    for name, param in model.backbone.named_parameters():
        param.requires_grad = False
    for name, param in model.preprocess.named_parameters():
        param.requires_grad = False
    for name, param in model.bbox_head.named_parameters():
        param.requires_grad = True
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # pdb.set_trace()
        batch_size = inputs.shape[0]
        outputs, outboxes = model(inputs)
        # loss = nn.mse_loss(outboxes, targets[1])
        loss = nn.l1_loss(outboxes, targets[1])
        iou = box_iou_batch(outboxes, targets[1])
        optimizer.step (loss)
        if batch_idx % 10 == 0:
            print(' BOX Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tBox_Acc: {:.6f}'.format(
                    epoch, batch_idx, len(train_loader),
                    100. * batch_idx / len(train_loader), loss.data[0], jt.sum(iou > 0.5)/batch_size))


total_acc = 0
total_num = 0
def test(model, val_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        batch_size = inputs.shape[0]
        outputs, outboxes = model(inputs)
        pred,_ = jt.argmax(outputs.data, dim=1)
        # pdb.set_trace()
        class_acc = jt.sum(targets[0] == pred)
        test_iou = box_iou_batch(outboxes, targets[1])
        all_correct = jt.sum((test_iou > 0.5)*(targets[0] == pred))
        global total_acc
        global total_num 
        total_acc += class_acc
        total_num += batch_size
        
    print('Test Epoch: {} [{}/{} ({:.0f}%)]\tClass Acc: {:.6f}\tBoth Acc: {:.6f}\tBox_Acc: {:.6f}'.format(epoch, \
                batch_idx, len(val_loader),100. * float(batch_idx) / len(val_loader), class_acc/ batch_size, all_correct/batch_size, jt.sum(test_iou > 0.5)/batch_size))
    return class_acc    

def param_dict():
    parser = ArgumentParser(description="Hyper parameters")
    parser.add_argument('-b','--batch_size', default=16, type=int)
    parser.add_argument('-l','--learning_rate', default=1e-2, type=float)
    parser.add_argument('-e','--epochs', default=200, type=int)
    args = parser.parse_args()
    return vars(args)

In [9]:
batch_size = 16
learning_rate = 1e-2
momentum = 0.9
weight_decay = 1e-4
epochs = 10

my_transform = trans.Compose([
    trans.ToTensor()
    ])

train_loader = Tiny_vid(train=True, transform=my_transform, aug=True).set_attrs(batch_size=batch_size, shuffle=True)
val_loader = Tiny_vid(train=False, transform=my_transform)
val_loader.set_attrs(batch_size=len(val_loader.ground_truth), shuffle=False)

model = DetNet()
optimizer = nn.SGD(list(filter(lambda val: val.requires_grad, model.parameters())), learning_rate, momentum, weight_decay)
optimizer_box = nn.SGD(list(filter(lambda val: val.requires_grad, model.parameters())), learning_rate, momentum, weight_decay)
for epoch in range(epochs):
    train(model, train_loader, optimizer, epoch)
    test(model, val_loader, epoch)

break: ./tiny_vid/bird    000151.JPEG
break: ./tiny_vid/car    000151.JPEG
break: ./tiny_vid/dog    000151.JPEG
break: ./tiny_vid/lizard    000151.JPEG
break: ./tiny_vid/turtle    000151.JPEG
dataset img len: 1500
dataset label len: 1500
break: ./tiny_vid/bird    000181.JPEG
break: ./tiny_vid/car    000181.JPEG
break: ./tiny_vid/dog    000181.JPEG
dataset img len: 150
dataset label len: 150



Compiling Operators(19/19) used: 2.31s eta:    0s 

Compiling Operators(7/7) used: 2.31s eta:    0s 

Compiling Operators(3/3) used: 2.31s eta:    0s 

Compiling Operators(4/4) used: 2.31s eta:    0s 


Train Epoch: 0 [0/94 (0%)]	Loss: 2.890447	Class Acc: 0.125000	Box_Acc: 0.000000	All Acc: 0.000000
